In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from code_utils.read_ipcc_bib import read_bib_wg
from code_utils.enriching_data_OpenAlex import get_countries_concepts_sdg,get_open_alex_data,get_open_alex_data_not_in_references
from code_utils.utils import aplatir,wg_chap_to_dict,get_doi_cleaned,check_doi_glutton,get_year_ipcc
from code_utils.pickle import load_cache,write_cache

In [4]:
cached_openalex_data = {}
cached_openalex_data_not_ipcc = {}

In [5]:
try:
    cached_openalex_data = load_cache(cached_openalex_data,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data.pkl')
    cached_openalex_data_not_ipcc = load_cache(cached_openalex_data_not_ipcc,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipcc.pkl')
except:
    #write_cache(cached_openalex_data,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data.pkl')
    #write_cache(cached_openalex_data_not_ipcc,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipcc.pkl')
    print('oups not in bibliography folder')

45597 data in cached openalex data
oups not in bibliography folder


In [6]:
#write_cache(cached_openalex_data,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data.pkl')
write_cache(cached_openalex_data_not_ipcc,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipcc.pkl')

Read IPCC references in .bib files

In [ ]:
wgs={'1':{'wg1':[], 'dataframes_1':{}, 'listdir1':['IPCC_AR6_WGI_References_Chapter01.bib',
 'IPCC_AR6_WGI_References_Chapter02.bib',
 'IPCC_AR6_WGI_References_Chapter03.bib',
 'IPCC_AR6_WGI_References_Chapter04.bib',
 'IPCC_AR6_WGI_References_Chapter05.bib',
 'IPCC_AR6_WGI_References_Chapter06.bib',
 'IPCC_AR6_WGI_References_Chapter07.bib',
 'IPCC_AR6_WGI_References_Chapter08.bib',
 'IPCC_AR6_WGI_References_Chapter09.bib',
 'IPCC_AR6_WGI_References_Chapter10.bib',
 'IPCC_AR6_WGI_References_Chapter11.bib',
 'IPCC_AR6_WGI_References_Chapter12.bib']},
     '2':{'wg2':[], 'wg2_update':[], 'dataframes_2':{}, 'listdir2':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG2")},
     '2_CROSS':{'wg2_CROSS':[], 'wg2_CROSS_update':[], 'dataframes_2_CROSS':{}, 'listdir2_CROSS':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG2_CROSS")},
     '3':{'wg3':[], 'dataframes_3':{}, 'listdir3':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG3")}}

In [ ]:
os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG1")

In [ ]:
for k in ['1','2','2_CROSS','3']:
    wgs[k][f'df_{k}'] = read_bib_wg(wgs,k,verbose=True)
    #wgs[k][f'df_{k}'].to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_wg{k}.jsonl', orient='records', lines=True)

Read and clean the data from 'structured_data' folder

In [38]:
data_all={}

In [39]:
for k in ['1','2','2_CROSS','3']:
   #data_all[f'df_wg{k}']=wgs[k][f'df_{k}']
   data_all[f'df_wg{k}']=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_wg{k}.jsonl', lines= True)

In [40]:
#all data 
df_ipcc=pd.concat(list(data_all.values()), ignore_index=True)

In [41]:
df_ipcc.loc[:,'doi']=df_ipcc.loc[:,'doi'].apply(lambda x: get_doi_cleaned(x))
df_ipcc.loc[:,'doi']=df_ipcc.loc[:,'doi'].apply(lambda x: None if str(x)[:4]=='http' else x)
df_ipcc['freq']=1

Enriching data through OpenAlex API integration

In [11]:
#df_unique_doi=df_ipcc.groupby(by=['doi','title','year'], dropna=True).agg({'wg': lambda x: list(x),'chap': lambda x: list(x)})
df_unique_doi=df_ipcc[['doi','title','year']].drop_duplicates(subset=['doi']).dropna(subset=['doi'])
df_unique_doi=df_unique_doi.reset_index()

In [12]:
#get data_OpenAlex from the API 
df_unique_doi.apply(lambda row: get_open_alex_data(cached_openalex_data,row['doi']), axis= 1)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=doi:10.1162/152638003763336374 HTTP/1.1" 200 3202
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=doi:10.1080/01441647.2017.1340234 HTTP/1.1" 200 2617
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=doi:10.1029/2019gh000208 HTTP/1.1" 200 5255
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?filter=doi:10.1007/s10584-018-2343-7 HTTP/1.1" 200 2954
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openalex.org:443
DEBUG:urllib3.connectionpool:https://api.openalex.org:443 "GET /works?f

0        [{'id': 'https://openalex.org/W4238825490', 'd...
1        [{'id': 'https://openalex.org/W640230686', 'do...
2        [{'id': 'https://openalex.org/W1994859565', 'd...
3        [{'id': 'https://openalex.org/W2038653220', 'd...
4        [{'id': 'https://openalex.org/W2767377409', 'd...
                               ...                        
54816                                                 None
54817                                                 None
54818                                                 None
54819                                                 None
54820                                                 None
Length: 54821, dtype: object

In [13]:
len(cached_openalex_data)

54821

In [14]:
write_cache(cached_openalex_data,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data.pkl')

In [15]:
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_list=[]
locations_id_list=[]
type_list=[]
type_crossref_list=[]

for i,row in df_unique_doi.iterrows():
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_id,type_OA,type_crossref=get_countries_concepts_sdg(cached_openalex_data,row)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_list.append(locations_names)
    locations_id_list.append(locations_id)
    type_list.append(type_OA)
    type_crossref_list.append(type_crossref)
    

df_unique_doi['countries'] = countries_list
df_unique_doi['concepts'] = concepts_list
df_unique_doi['sdg'] = sdg_list
df_unique_doi['topics'] = topics_list
df_unique_doi['is_OA_available'] = is_OA_available_list
df_unique_doi['title_OA'] = title_list
df_unique_doi['year_OA'] = year_list
df_unique_doi['authors_name'] = names_list
df_unique_doi['rors'] = rors_list
df_unique_doi['institutions_names'] = institutions_names_list
df_unique_doi['locations_names'] = locations_list
df_unique_doi['locations_ids'] = locations_id_list
df_unique_doi['type'] = type_list
df_unique_doi['type_crossref'] = type_crossref_list

In [16]:
df_unique_doi.locations_ids[0]

'1664-8714'

In [17]:
#df_unique_doi['wg_chap']=df_unique_doi.apply(lambda row: wg_chap_to_dict(row), axis=1)
df_unique_doi.loc[:,'countries']=df_unique_doi.loc[:,'countries'].apply(lambda x: str(x).replace('nan','None').replace('[]','None').replace('[None]','None') if ((str(x)=='nan')|(str(x)=='[None]')|((str(x)=='[]'))) else x )
df_unique_doi.loc[:,'countries']=df_unique_doi.loc[:,'countries'].apply(lambda x: ['None'] if x=='None' else x)
df_unique_doi.loc[:,'year']=df_unique_doi.loc[:,'year'].apply(lambda x: get_year_ipcc(x))
df_unique_doi['test_glutton']=df_unique_doi.apply(lambda row: check_doi_glutton(row), axis=1)
df_unique_doi=df_unique_doi[(df_unique_doi['test_glutton']==True)&(pd.isna(df_unique_doi['is_OA_available'])==False)]

In [21]:
df_unique_doi

,index,doi,title,year,countries,concepts,sdg,topics,is_OA_available,title_OA,year_OA,authors_name,rors,institutions_names,locations_names,locations_ids,type,type_crossref,test_glutton
0,0,10.3389/978-2-88963-118-6,{OceanObs'19: An Ocean of Opportunity. Volume 1},2019,[None],"[Volume (thermodynamics), Oceanography, Enviro...","[{'id': '14', 'name': 'Life below water'}]",[Marine Biology and Ecology Research],True,Oceanobs'19: An Ocean of Opportunity. Volume I,2020.0,[None],[None],[None],Frontiers research topics,1664-8714,book,book,True
1,1,10.1007/978-94-015-9612-1,"{Eduard Br{\""{u}}ckner – The Sources and Conse...",2000,"[CA, UA]","[Geographer, Climate change, Politics, Climato...","[{'id': '13', 'name': 'Climate action'}]","[Geology and Paleoclimatology Research, Climat...",True,Eduard Brückner — The Sources and Consequences...,2000.0,"[(Nico Stehr, [CA]), (Hans von Storch, [UA])]","[(https://ror.org/03rmrcq20, CA), (https://ror...","[(University of British Columbia, CA), (Instit...",None,None,book,book,True
2,4,10.1002/andp.19003081208,"{{\""{U}}ber die Bedeutung des Wasserdampfes un...",1900,[US],"[Physics, Humanities, Angstrom, Crystallograph...",None,"[Atmospheric Ozone and Climate, Atmospheric ch...",True,Ueber die Bedeutung des Wasserdampfes und der ...,1900.0,"[(Knut Ångström, [US])]","[(https://ror.org/03wbhj477, US)]","[(Upsala College, US)]",Annalen der Physik,0003-3804,article,journal-article,True
3,5,10.3402/tellusa.v16i1.8885,{The parameters of atmospheric turbidity},1964,[US],"[Turbidity, Environmental science, Meteorology...",None,[Ocean Waves and Remote Sensing],True,The parameters of atmospheric turbidity,1964.0,"[(Anders Ångström, [US])]","[(https://ror.org/03a8x4b06, US)]","[(Eppley Foundation for Research, US)]",Tellus A Dynamic Meteorology and Oceanography,0280-6495,article,journal-article,True
4,6,10.1080/20014422.1929.11880498,{On the Atmospheric Transmission of Sun Radiat...,1929,[None],"[Atmospheric dust, Radiation, Transmission (te...","[{'id': '14', 'name': 'Life below water'}]",[Atmospheric aerosols and clouds],True,On the Atmospheric Transmission of Sun Radiati...,1929.0,"[(Anders Ångström, [])]",[],[],Geografiska Annaler,1651-3215,article,journal-article,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54815,81207,10.1016/j.worlddev.2020.105249,China's green transformation through eco-indus...,2021.0,"[DE, US, CN]","[China, Business, Industrial ecology, Material...",None,"[Sustainable Industrial Ecology, Sustainable S...",True,China’s green transformation through eco-indus...,2020.0,"[(Douglas Zhihua Zeng, [US]), (Lei Cheng, [CN]...","[(https://ror.org/00ae7jd04, US), (https://ror...","[(World Bank, US), (State Key Joint Laboratory...",World Development,0305-750X,article,journal-article,True
54817,81209,10.1016/j.jclepro.2018.05.194,"Water-energy-food nexus: Concepts, questions a...",2018.0,"[GB, CN]","[Nexus (standard), Management science, Water e...","[{'id': '12', 'name': 'Responsible consumption...","[Water-Energy-Food Nexus Studies, Energy Harve...",True,"Water-energy-food nexus: Concepts, questions a...",2018.0,"[(Chi Zhang, [CN]), (Xiaoxian Chen, [CN]), (Yu...","[(https://ror.org/023hj5876, CN), (https://ror...","[(Dalian University of Technology, CN), (Dalia...",Journal of Cleaner Production,0959-6526,article,journal-article,True
54818,81210,10.1016/j.apenergy.2017.07.036,The water-food-energy nexus optimization appro...,2018.0,"[SE, US]","[Nexus (standard), Agriculture, Yield (enginee...",None,"[Water-Energy-Food Nexus Studies, Water resour...",True,The water-food-energy nexus optimization appro...,2017.0,"[(Jie Zhang, [US]), (Pietro Elia Campana, [SE]...","[(https://ror.org/047s2c258, US), (https://ror...","[(University of Maryland, College Park, US), (...",Applied Energy,0306-2619,article,journal-article,True
54819,81212,10.1016/j.enpol.2013.08.092,To what extent does wind power deployment affe...,2013.0,[CN],"[Software deployment, Revenue, Renewabl

In [23]:
df_unique_doi.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_ipcc.jsonl', orient='records', lines= True)

In [42]:
df_ipcc=df_ipcc[['year', 'url', 'doi','title','author','freq']]

In [43]:
df_ipcc=pd.merge(df_ipcc, df_unique_doi[['doi','countries','concepts', 'sdg', 'topics', 'is_OA_available', 'title_OA', 'year_OA', 'authors_name', 'rors', 'institutions_names', 'locations_names', 'locations_ids', 'type', 'type_crossref', 'test_glutton']], on='doi', how='left')
df_ipcc.loc[:,'countries']=df_ipcc.loc[:,'countries'].apply(lambda x: str(x).replace('nan','None').replace('[]','None').replace('[None]','None') if ((str(x)=='nan')|(str(x)=='[None]')|((str(x)=='[]'))) else x )
df_ipcc.loc[:,'countries']=df_ipcc.loc[:,'countries'].apply(lambda x: ['None'] if x=='None' else x)

In [44]:
df_ipcc.loc[:,'year']=df_ipcc.loc[:,'year'].apply(lambda x: get_year_ipcc(x))

In [45]:
df_ipcc['test_glutton']=df_ipcc.apply(lambda row: check_doi_glutton(row), axis=1)

In [46]:
df_ipcc.drop_duplicates(subset=['doi']).test_glutton.value_counts()

test_glutton
True     52471
False     2351
Name: count, dtype: int64

In [47]:
df_ipcc=df_ipcc[(df_ipcc['test_glutton']==True)&(pd.isna(df_ipcc['is_OA_available'])==False)]

In [48]:
df_ipcc.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json', orient='records')

In [56]:
df_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json').dropna(subset=['year'])

In [65]:
pd.Series(aplatir(list(df_ipcc.dropna(subset=['topics']).topics))).value_counts()[:50]

Climate variability and models                         13109
Climate Change Policy and Economics                     5697
Climate change impacts on agriculture                   5092
Meteorological Phenomena and Simulations                5062
Climate Change, Adaptation, Migration                   4760
Atmospheric and Environmental Gas Dynamics              4600
Sustainability and Climate Change Governance            3783
Flood Risk Assessment and Management                    3445
Cryospheric studies and observations                    3422
Climate Change and Health Impacts                       3307
Geology and Paleoclimatology Research                   3299
Tropical and Extratropical Cyclones Research            3093
Plant Water Relations and Carbon Dynamics               2750
Coral and Marine Ecosystems Studies                     2601
Oceanographic and Atmospheric Processes                 2578
Atmospheric chemistry and aerosols                      2521
Environmental Impact and

In [ ]:
climat_topics=['climate change','ecological','climate variability','flood risk assessment','environmental gas dynamics','cryospheric studies','paleoclimatology','cyclones research','plant water relations','carbon dynamics','marine ecosystems','sustainability','atmospheric processes','atmospheric chemistry','disaster management and resilience']

Find the data for constructing the learning model ( IPCC related or not )

In [49]:
df_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_ipcc.jsonl', lines= True).dropna(subset=['year'])

In [50]:
df_ipcc=df_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids'])

In [51]:
len(df_ipcc)

48101

In [52]:
df_ipcc['year'] = pd.to_numeric(df_ipcc['year'], errors='coerce')
year=df_ipcc.drop_duplicates(subset='doi')['year'].dropna().sort_values().apply(lambda x: int(x))
year_counts = pd.Series(list(year)).value_counts().to_dict()

In [53]:
sum(list(year_counts.values()))

48101

In [54]:
len(aplatir(list(cached_openalex_data_not_ipcc.values())))

0

In [ ]:
dois=list(df_ipcc.doi.dropna().drop_duplicates())
year_counts_not_ipcc={}

In [ ]:
for year in list(year_counts.keys()):   
    cached_openalex_data_not_ipcc[year]=[]
    year_counts_not_ipcc[year]=0
    while year_counts_not_ipcc[year]<year_counts[year]:
        get_open_alex_data_not_in_references(dois,cached_openalex_data_not_ipcc,year_counts,year_counts_not_ipcc,year)
    cached_openalex_data_not_ipcc[year]=cached_openalex_data_not_ipcc[year][:year_counts[year]]

In [ ]:
for year in list(year_counts.keys()):   
    cached_openalex_data_not_ipcc[year]=cached_openalex_data_not_ipcc[year][:year_counts[year]]

In [ ]:
write_cache(cached_openalex_data_not_ipcc,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipcc.pkl')

In [ ]:
df_not_ipcc=pd.DataFrame()
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_list=[]
doi_list=[]
locations_id_list=[]


for k in range(len(aplatir(list(cached_openalex_data_not_ipcc.values())))):
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_id=get_countries_concepts_sdg(cached_openalex_data=aplatir(list(cached_openalex_data_not_ipcc.values())),ipcc=False,i=k)

    doi_list.append(doi)
    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_list.append(locations_names)
    locations_id_list.append(locations_id)
    
df_not_ipcc['doi'] = doi_list
df_not_ipcc['countries'] = countries_list
df_not_ipcc['concepts'] = concepts_list
df_not_ipcc['sdg'] = sdg_list
df_not_ipcc['topics'] = topics_list
df_not_ipcc['is_OA_available'] = is_OA_available_list
df_not_ipcc['title'] = title_list
df_not_ipcc['year'] = year_list
df_not_ipcc['authors_name'] = names_list
df_not_ipcc['rors'] = rors_list
df_not_ipcc['institutions_names'] = institutions_names_list
df_not_ipcc['locations_names'] = locations_list
df_not_ipcc['locations_ids'] = locations_id_list

In [ ]:
len(df_not_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids']))

In [ ]:
len(df_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids']))

In [ ]:
df_not_ipcc.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_not_ipcc.jsonl', orient='records', lines= True)